Replicate EV/REV plots for Octopamine/Rolipram sessions previously generated by Jahangir.

In [1]:
# Import all the functions you need
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
import os
from pathlib import Path
import platform

# Comment here to add in for either mac or linux computer
if platform.system() == "Linux":   # for linux
    sys.path.extend(['/data/GitHub/NeuroPy']) 
    sys.path.extend(['/data/GitHub/TraceFC/'])
elif platform.system() == "Darwin":
    sys.path.extend(['/Users/nkinsky/Documents/UM/GitHub/NeuroPy'])
    sys.path.extend(['/Users/nkinsky/Documents/UM/GitHub/TraceFC'])
elif platform.system() == "Windows":
    print('No windows repositories defined')

from neuropy.core.neurons import Neurons
from neuropy.analyses.reactivation import ExplainedVariance
from neuropy.core.epoch import Epoch
from neuropy.core.session import ProcessData

plot_save_dir = Path('/home/nkinsky/Dropbox (University of Michigan)/Shared_Folders/KD/Grants/R01_octopamine_rollipram/Nat_plots')

/home/nkinsky/miniforge3/envs/NeuroPy/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


### Get session info

In [129]:
sesh_csv = "/data3/Octopamine_Rolipram/Recording_Rats/OCT_ROL_sessions.csv"
sesh_df = pd.read_csv(sesh_csv, header=0)
sesh_df['session'] = pd.Categorical(sesh_df['session'], ['PBS', 'SAL', 'ROL', 'OCT'])
sesh_df = sesh_df.sort_values(["Animal", "session"]).reset_index().drop(columns = "index")
sesh_df

,Animal,session,date,directory,processed
0,A11,PBS,2019_12_02,/data3/Octopamine_Rolipram/Recording_Rats/A11/...,N
1,A11,ROL,2019_12_12,/data3/Octopamine_Rolipram/Recording_Rats/A11/...,N
2,A11,OCT,2019_11_30,/data3/Octopamine_Rolipram/Recording_Rats/A11/...,N
3,A14,PBS,2020_02_23,/data3/Octopamine_Rolipram/Recording_Rats/A14/...,Y
4,A14,ROL,2020_02_26,/data3/Octopamine_Rolipram/Recording_Rats/A14/...,Y
5,A14,OCT,2020_02_21,/data3/Octopamine_Rolipram/Recording_Rats/A14/...,Y
6,BG,SAL,2019_10_21,/data3/Octopamine_Rolipram/Recording_Rats/BG/B...,Y
7,BG,ROL,2019_10_19,/data3/Octopamine_Rolipram/Recording_Rats/BG/B...,Y
8,BG,OCT,None,None,N
9,BGU,SAL,2021_08_11,/data3/Octopamine_Rolipram/Recording_Rats/BGU/...,Y


In [7]:
df_use = sesh_df[(sesh_df.Animal == "A11") & (sesh_df.session == "OCT")]
df_use

,Animal,session,date,directory
2,A11,OCT,None,/data3/Octopamine_Rolipram/Recording_Rats/A11/...


In [167]:
neurons_new.get_

Neurons
 n_neurons: 0
 t_start: 0.0
 t_stop: 52134
 neuron_type: ['mua']

In [173]:
neurons_new.neuron_type == 'mua'

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True, False,
       False,  True,  True,  True, False,  True,  True,  True,  True,
        True])

### Loop through and update everything

In [172]:
for sesh_name in ["OCT", "PBS", "ROL"]:
    df_use = sesh_df[(sesh_df.Animal == "A11") & (sesh_df.session == sesh_name)]
    working_dir = Path(df_use.directory.values[0])
    
    # Get XML file name
    xmlfile = sorted(working_dir.glob("*.xml"))[0]

    # Build up epochs files
    blocks_dict = np.load(sorted(working_dir.glob("**/*_epochs.npy"))[0], allow_pickle=True).item()
    blocks_dict
    starts = [block[0] for block in blocks_dict.values()]
    stops = [block[1] for block in blocks_dict.values()]
    labels = list(blocks_dict.keys())
    epoch_df = pd.DataFrame({"start": starts, "stop": stops, "label": labels})
    epoch_df.loc[epoch_df.label == "run", "label"] = "maze"
    epoch_df.loc[epoch_df.label == "rs", "label"] = "post"
    epochs = Epoch(epochs=epoch_df)
    epochs.save(xmlfile.with_suffix(".epoch.npy"))

    # Load in basics
    basics = np.load(sorted(working_dir.glob("archive/*_basics.npy"))[0], allow_pickle=True).item()

    # Load in old-style neurons file
    neurons_old = Neurons.from_file(sorted(working_dir.glob("**/*_spikes.npy*"))[0])

    # Build up waveforms - make it zero everywhere except for the shank in question
    basics = np.load(sorted(working_dir.glob("archive/*_basics.npy"))[0], allow_pickle=True).item()
    nchan = basics['nChans']
    nneurons = len(neurons_old['templates'])
    nbins = neurons_old['templates'][0].shape[1]
    waveforms = []
    for i in range(nneurons):
        waveform = np.zeros((nchan, nbins))
        shank = neurons_old['info'].loc[i, "shank"].astype(int)
        shank_channels = basics['channelgroups'][shank - 1]
        try:
            waveform[np.min(shank_channels): (np.max(shank_channels) + 1)] = neurons_old['templates'][i]
        except ValueError:
            nchan_on_shank = neurons_old['templates'][i].shape[0]
            waveform[np.min(shank_channels): (np.min(shank_channels) + nchan_on_shank)]
            print(f"Channel # mismatch for unit {i} on shank {shank}! Be sure to check. Hacking it for now.")
            
        waveforms.append(waveform)

    # Build up new neurons file and save
    neurons_new = Neurons(neurons_old["times"], t_stop = epochs['post'].stops[0], neuron_type=neurons_old["info"].group.values,
                          waveforms=np.array(waveforms))
    neurons_new.save(xmlfile.with_suffix(".neurons.npy"))

/data3/Octopamine_Rolipram/Recording_Rats/A11/A11_2019-11-30_SDOCT/A11_2019-11-30_SDOCT.epoch.npy saved
/data3/Octopamine_Rolipram/Recording_Rats/A11/A11_2019-11-30_SDOCT/A11_2019-11-30_SDOCT.neurons.npy saved
/data3/Octopamine_Rolipram/Recording_Rats/A11/A11_2019-12-02_SDPBS/A11_2019-12-02_SDPBS.epoch.npy saved
Channel # mismatch for unit 3 on shank 2! Be sure to check. Hacking it for now.
Channel # mismatch for unit 4 on shank 2! Be sure to check. Hacking it for now.
/data3/Octopamine_Rolipram/Recording_Rats/A11/A11_2019-12-02_SDPBS/A11_2019-12-02_SDPBS.neurons.npy saved
/data3/Octopamine_Rolipram/Recording_Rats/A11/A11_2019-12-12_SDROL/A11_2019-12-12_SDROL.epoch.npy saved
/data3/Octopamine_Rolipram/Recording_Rats/A11/A11_2019-12-12_SDROL/A11_2019-12-12_SDROL.neurons.npy saved
